In [ ]:
# Import the required packages
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
from pyspark.sql.functions import regexp_replace, initcap
from dotenv import load_dotenv
import os

load_dotenv()

In [ ]:
# Connect Azure Databricks
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "client_id",
    "fs.azure.account.oauth2.client.secret": 'secret_key',
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenant_id/oauth2/token"
}

dbutils.fs.updateMount(
    source="abfss://tokyo-olympica-data@tokyoolympicadatarm1.dfs.core.windows.net",
    mount_point="/mnt/tokyodata",
    extra_configs=configs
)
  

Out[87]: True

In [ ]:
# Check that the mounting was successful
%fs
ls "/mnt/tokyodata"

path,name,size,modificationTime
dbfs:/mnt/tokyodata/raw-data/,raw-data/,0,1721115071000
dbfs:/mnt/tokyodata/transformed-data/,transformed-data/,0,1721115446000


In [ ]:
# Load the files from raw-data folder
athletes = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyodata/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyodata/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyodata/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyodata/raw-data/medals.csv")
teams = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyodata/raw-data/teams.csv")

In [ ]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [ ]:
# Rename PersonName to Athlete Name & have the name in proper format
athletes = athletes.withColumn("PersonName",initcap(regexp_replace("`PersonName`", r'^\s+|\s+$', '')))\
    .withColumnRenamed("PersonName", "Athlete_Name")
    
display(athletes)

Athlete_Name,Country,Discipline
Aalerud Katrine,Norway,Cycling Road
Abad Nestor,Spain,Artistic Gymnastics
Abagnale Giovanni,Italy,Rowing
Abalde Alberto,Spain,Basketball
Abalde Tamara,Spain,Basketball
Abalo Luc,France,Handball
Abaroa Cesar,Chile,Rowing
Abass Abobakr,Sudan,Swimming
Abbasali Hamideh,Islamic Republic of Iran,Karate
Abbasov Islam,Azerbaijan,Wrestling


In [ ]:
athletes.printSchema()

root
 |-- Athlete_Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [ ]:
coaches.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    null|
|     

In [ ]:
# Rename PersonName to Athlete Name & have the name in proper format
coaches = coaches.withColumn("Name",initcap(regexp_replace("`Name`", r'^\s+|\s+$', '')))\
    .withColumnRenamed("Name", "Coach_Name")
    
display(coaches)

Coach_Name,Country,Discipline,Event
Abdelmagid Wael,Egypt,Football,null
Abe Junya,Japan,Volleyball,null
Abe Katsuhiko,Japan,Basketball,null
Adama Cherif,C�te d'Ivoire,Football,null
Ageba Yuya,Japan,Volleyball,null
Aikman Siegfried Gottlieb,Japan,Hockey,Men
Al Saadi Kais,Germany,Hockey,Men
Alameda Lonni,Canada,Baseball/Softball,Softball
Alekno Vladimir,Islamic Republic of Iran,Volleyball,Men
Alekseev Alexey,ROC,Handball,Women


In [ ]:
coaches.printSchema()

root
 |-- Coach_Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
entriesgender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [ ]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [ ]:
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [ ]:
# Rename Team_Country to Country
medals = medals.withColumnRenamed("Team_Country", "Country")\
    .withColumnRenamed("Rank by Total", "Rank_by_Total")

In [ ]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank_by_Total: integer (nullable = true)



In [ ]:
teams.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [ ]:
# Rename TeamName to 'Team Name'
teams = teams.withColumnRenamed("TeamName", "Team_Name")

In [ ]:
teams.printSchema()

root
 |-- Team_Name: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
# Find the top countries with the highest number of medals
top_medal_countries = medals.orderBy("Gold", "Silver", "Bronze", ascending=False).select("Country", "Gold", "Silver", "Bronze").show()


+--------------------+----+------+------+
|             Country|Gold|Silver|Bronze|
+--------------------+----+------+------+
|United States of ...|  39|    41|    33|
|People's Republic...|  38|    32|    18|
|               Japan|  27|    14|    17|
|       Great Britain|  22|    21|    22|
|                 ROC|  20|    28|    23|
|           Australia|  17|     7|    22|
|         Netherlands|  10|    12|    14|
|              France|  10|    12|    11|
|             Germany|  10|    11|    16|
|               Italy|  10|    10|    20|
|              Canada|   7|     6|    11|
|              Brazil|   7|     6|     8|
|         New Zealand|   7|     6|     7|
|                Cuba|   7|     3|     5|
|             Hungary|   6|     7|     7|
|   Republic of Korea|   6|     4|    10|
|              Poland|   4|     5|     5|
|      Czech Republic|   4|     4|     3|
|               Kenya|   4|     4|     2|
|              Norway|   4|     2|     2|
+--------------------+----+------+

In [ ]:
# Write and load data
athletes.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/tokyodata/transformed-data/athletes.csv")
coaches.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/tokyodata/transformed-data/coaches.csv")
entriesgender.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/tokyodata/transformed-data/entriesgender.csv")
medals.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/tokyodata/transformed-data/medals.csv")
teams.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/tokyodata/transformed-data/teams.csv")
